# 超音波センサーチェック　ｃｈ８

各チャンネルＥｃｈｏ端子は、入力電圧範囲は２．９０Ｖ～３．４０Ｖ内であること

In [1]:
import Jetson.GPIO as GPIO
import time
from IPython.display import clear_output  # Jupyter Lab用

# GPIOのモードをボード物理ピンに設定
GPIO.setmode(GPIO.BOARD)
GPIO.setwarnings(False)

# 各HC-SR04センサーのトリガーとエコーピンをリストで定義
# ※ 下記のGPIO番号は一例です。実際の配線に合わせて変更してください。
sensors = [
    {"trigger": 12, "echo": 11},
    {"trigger": 16, "echo": 13},
    {"trigger": 18, "echo": 15},
    {"trigger": 22, "echo": 29},
    {"trigger": 32, "echo": 31},
    {"trigger": 36, "echo": 33},
    {"trigger": 38, "echo": 35},
    {"trigger": 40, "echo": 37}
]

# 各センサーのGPIOピンのセットアップ
for sensor in sensors:
    GPIO.setup(sensor["trigger"], GPIO.OUT)
    GPIO.setup(sensor["echo"], GPIO.IN)
    # トリガーは初期状態LOWにしておく
    GPIO.output(sensor["trigger"], False)

def measure_distance(trigger_pin, echo_pin):
    """
    指定されたトリガーとエコーピンを用いて距離を計測する関数
    """
    # トリガーを一度LOWにしてセンサーを安定させる
    GPIO.output(trigger_pin, False)
    time.sleep(0.0002)  # 200マイクロ秒待機

    # トリガーに10マイクロ秒のパルスを送信
    GPIO.output(trigger_pin, True)
    time.sleep(0.00001)  # 10マイクロ秒
    GPIO.output(trigger_pin, False)

    # エコーがHIGHになるのを待つ（パルス開始時間）
    pulse_start = time.time()
    timeout = pulse_start + 0.04  # 40msタイムアウト
    while GPIO.input(echo_pin) == 0 and time.time() < timeout:
        pulse_start = time.time()

    # エコーがLOWになるのを待つ（パルス終了時間）
    pulse_end = time.time()
    timeout = pulse_end + 0.04  # 40msタイムアウト
    while GPIO.input(echo_pin) == 1 and time.time() < timeout:
        pulse_end = time.time()

    # パルスの時間（往復時間）
    pulse_duration = pulse_end - pulse_start

    # 距離の計算（音速=34300 cm/s、往復なので2で割る）
    distance = pulse_duration * 17150  # (34300/2) = 17150
    distance = round(distance, 2)
    return distance

try:
    while True:
        clear_output(wait=True)
        # 各センサーごとに距離を計測
        for i, sensor in enumerate(sensors):
            distance = measure_distance(sensor["trigger"], sensor["echo"])
            print(f"センサー {i+1}: {distance} cm")
        print("-" * 30)
        time.sleep(0.3)  # 1秒間隔で更新

except KeyboardInterrupt:
    print("測定を停止します")
    GPIO.cleanup()


センサー 1: 36.1 cm
センサー 2: 48.18 cm
センサー 3: 68.14 cm
センサー 4: 16.63 cm
センサー 5: 16.37 cm
センサー 6: 17.59 cm
センサー 7: 42.13 cm
センサー 8: 27.85 cm
------------------------------
測定を停止します
